In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy.linalg import orth
from scipy.linalg import norm

class dog_leg:
    def __init__(self, f, df, d2f):
        self.f = f
        self.df = df
        self.d2f = d2f


    def dogleg_step(self, x_k, trust_radius_Delta):
        '''
        Subspace trust-region solver for quadratic model subproblem
        Inputs:
        f: function handler
        df: gradient handler
        d2f: Hessian handler
        x_k: current iterate
        Trust_radius_Delta: Max trust-region radius
        
        Outputs:
        p: step (direction times length)
        '''

        g = self.df(x_k)
        B = self.d2f(x_k)

        # Compute Newton Direction
        a = -np.linalg.solve(B, g) #-B/g
        # det_B = np.linalg.det(B)
        # if det_B == 0:
        #     a = np.zeros(shape=(np.shape(g)))
        # else:
        #     a = -np.dot(np.linalg.inv(B), g) 
        #     print('2')

        # Orthonormal basis to check for collinearity
        V = orth([g,-a])
        
        # Case 1 - compute unconstrained solution and check if its in the trust region
        if a.T @ a < trust_radius_Delta**2:
            p = a
            return p
        
        # Case 2
        # If newton point p_B is outside TR, but collinear to gradient
        if a.T @ a >= trust_radius_Delta**2 and np.shape(V)[1] == 1:
            # calc cauchy point
            gTBg = g.T @ (B @ g)
            if gTBg <= 0:
                tau = 1
            else:
                tau = min(norm(g)**3 / (trust_radius_Delta * gTBg), 1)
            p = -tau * (trust_radius_Delta) / norm(g) @ g
            return p

        p_U = (-g @ g / (g.T @ B @ g)) @ g
        
        # Case 3
        # If newton point p_B is inside TR, but p_U is outside TR
        if norm(p_U) >= trust_radius_Delta:
            p = (trust_radius_Delta / norm(p_U)) * p_U
            return p
        
        # Case 4
        # If newton point p_B is outside TR, but p_U is inside TR
        if norm(p_U) < trust_radius_Delta:
            # find tau that solces 
            # || p_U + tau(p_B - p_U) ||**2 = Delta_k**2
            tau2 = Functions.root_finder(p_U, a, trust_radius_Delta, t=0.5)
            # we put that tau back into below to find intersect point
            p = (p_U + (tau2 - 1) * (a - p_U))
            return p
        

    def _minimize_trust_region(self, fun, x0, args=(), subproblem=DoglegSubproblem, initial_trust_radius=3.0,
                        max_trust_radius=1000.0, eta=0.15, gtol=1e-5, maxiter=None, disp=True, return_all=True, **unknown_options):
        #the trust region method to solve rosenbrock function with Dogleg sub problem implementation with trust radius
        #and call back enabled
        #wrap fucntion
        nfun, fun = wrap_function(fun, args)
        njac, jac = wrap_function(jac, args)
        nhess, hess = wrap_function(hess, args)
        nhessp, hessp = wrap_function(hessp, args)

        # limiting the number of iterations
        if maxiter is None:
            maxiter = len(x0)*200

        # init the search status
        warnflag = 0

        # initializing the search
        info = {'deltas': [], 'xs':[], 'iters': []}
        trust_radius = initial_trust_radius
        x = x0
        if return_all:
            allvecs = [x]
        m = subproblem(x, fun, jac, hess, hessp)#call to the dogleg sub problem
        k = 0

        # search for the function min
        while True:
            try:
                p, hits_boundary = m.solve(trust_radius)
            except np.linalg.linalg.LinAlgError as e:
                warnflag = 3
                break

            predicted_value = m(p)#predicted value at the proposed point

            #proposed point
            x_proposed = x + p
            m_proposed = subproblem(x_proposed, fun, jac, hess, hessp)

            #calculating the ratio based on actual reduction and predicted reduction
            actual_reduction = m.fun - m_proposed.fun
            predicted_reduction = m.fun - predicted_value
            if predicted_reduction <= 0:
                warnflag = 2
                break
            rho = actual_reduction / predicted_reduction

            # updating the trust radius based on rho value
            if rho < 0.25:
                trust_radius *= 0.25
            elif rho > 0.75 and hits_boundary:
                trust_radius = min(2*trust_radius, max_trust_radius)

            # if the ratio is high accept the proposed step
            if rho > eta:
                x = x_proposed
                m = m_proposed

            # if return_all:
            if callback is not None:
                callback(x)#callback to trace the values at the current iteration

            k += 1
            info['xs'].append(x)
            info['deltas'].append(trust_radius)
            info['iters'].append(k)

            # Stopping condition based on tolerance value
            if m.jac_mag < gtol:
                warnflag = 0
                break

            # Stopping conditions based on maximum number of iterations
            if k >= maxiter:
                warnflag = 1
                break

                # Default status messages
        status_messages = (
            _status_message['success'],
            _status_message['maxiter'],
            'A bad approximation caused failure to predict improvement.',
            'A linalg error occurred, such as a non-psd Hessian.',
        )

        #Scipy display options if no warning
        if disp:
            if warnflag == 0:
                print(status_messages[warnflag])
            else:
                print('Warning: ' + status_messages[warnflag])
            # print("         Current function value: %f")
            print("         Iterations: %d" % k)
            print("         Function evaluations: %d" % nfun[0])
            print("         Gradient evaluations: %d" % njac[0])
            print("         Hessian evaluations: %d" % nhess[0])

        result = Result(x=x, success=(warnflag == 0), status=warnflag, fun= opt.rosen,
                        jac=m.jac, nfev=nfun[0], njev=njac[0], nhev=nhess[0],
                        nit=k, message=status_messages[warnflag])
        return info

    def _minimize_dogleg(fun, x0, args=(), jac=jac, hess=hess,
                        **trust_region_options):
        return _minimize_trust_region(fun, x0, args=args, jac=jac, hess=hess,
                                    subproblem=DoglegSubproblem,
                                    **trust_region_options)

        # #Dogleg Sub problem
        # class DoglegSubproblem(BaseQuadraticSubproblem):
        # def cauchy_point(self):
        #     if self._cauchy_point is None:
        #         g = self.jac
        #         Bg = self.hess @ g
        #         self._cauchy_point = -(np.dot(g, g) / np.dot(g, Bg)) * g #defining the cauchypoint
        #     return self._cauchy_point

        # def newton_point(self):
        #     if self._newton_point is None:
        #         g = self.jac
        #         B = self.hess
        #         self._newton_point = -np.linalg.inv(B).dot(g)
        #     return self._newton_point

        # def solve(self, trust_radius):
        #     p_best = self.newton_point()
        #     if scipy.linalg.norm(p_best) < trust_radius:
        #         hits_boundary = False
        #         return p_best, hits_boundary

        #     # Compute the p_U 
        #     g = self.jac
        #     Bg = self.hess @ g
        #     p_u = -(np.dot(g, g) / np.dot(g, Bg)) * g 

        #     # If the p_U and p_B points are outside the trust region
        #     # return the point where the path intersects the boundary.
        #     p_u_norm = scipy.linalg.norm(p_u)
        #     if p_u_norm >= trust_radius:
        #         p_boundary = p_u * (trust_radius / p_u_norm)
        #         hits_boundary = True
        #         return p_boundary, hits_boundary


        #     _, tb = self.get_boundaries_intersections(p_u, p_best - p_u,
        #                                                 trust_radius)
        #     p_boundary = p_u + tb * (p_best - p_u)
        #     hits_boundary = True
        #     return p_boundary, hits_boundary